In [9]:
library(rtweet)
library(tidyverse)

In [10]:
set_renv_token <- function(path_to_token, override = FALSE) {
    ## check path
    stopifnot(
        is.character(path_to_token),
        file.exists(path_to_token)
    )
    ## expand to full path
    path_to_token <- normalizePath(path_to_token)

    ## store path to .Renviron
    renv <- normalizePath("~/.Renviron")
    
    ## if override = false and there's already a TWITTER_PAT, stop
    ## else override and there's already a TWITTER_PAT, then drop TWITTER_PAT and
    ## save new .Renviron
    if (!override && !identical(Sys.getenv("TWITTER_PAT"), "")) {
        stop("There's already a TWITTER_PAT. Use `override = TRUE` to replace.",
            call. = FALSE)
    } else if (!identical(Sys.getenv("TWITTER_PAT"), "") && 
               file.exists(renv)) {
        con <- file(renv)
        x <- readLines(con, warn = FALSE)
        close(con)
        x <- grep("^TWITTER_PAT", x, invert = TRUE, value = TRUE)
        writeLines(x, renv)
    }
    
    ## create env variable TWITTER_PAT (with path to saved token)
    envvar <- paste0("TWITTER_PAT=", path_to_token)
    
    ## save as .Renviron file (or append if the file already exists)
    cat(envvar, file = renv, fill = TRUE, append = TRUE)
}

In [19]:
set_renv_token("C:/Users/Jacky Zhao/token.rds", override = TRUE)

In [20]:
Sys.getenv("TWITTER_PAT")

[1] "C:UsersJacky ZhaoDocuments/.rtweet_token1.rds"

In [12]:
packageVersion("rtweet")

[1] '0.6.0'

In [13]:
## search for users
foxnews <- search_users("fox news", n = 1, verbose = FALSE)
msnbc <- search_users("msnbc", n = 1, verbose = FALSE)
cnn <- search_users("cnn", n = 1, verbose = FALSE)

In [14]:
head(cnn, 2)

code,message
32,Could not authenticate you.


In [15]:
head(msnbc)

code,message
32,Could not authenticate you.


In [32]:
head(foxnews)

user_id,name,screen_name,location,description,url,protected,followers_count,friends_count,listed_count,statuses_count,favourites_count,account_created_at,verified,profile_url,profile_expanded_url,account_lang,profile_banner_url,profile_background_url,profile_image_url
1367531,Fox News,FoxNews,U.S.A.,"America’s Strongest Primetime Lineup Anywhere! Follow America's #1 cable news network, delivering you breaking news, insightful analysis, and must-see videos.",http://t.co/ZYG58XZtAC,FALSE,17111156,404,62282,359333,0,2007-03-17 19:01:26,TRUE,http://t.co/ZYG58XZtAC,http://www.foxnews.com,en,https://pbs.twimg.com/profile_banners/1367531/1492649996,http://pbs.twimg.com/profile_background_images/414488415/xyz123abc.jpg,http://pbs.twimg.com/profile_images/918480715158716419/4X8oCbge_normal.jpg


In [33]:
## get followers
dem <- get_followers("msnbc", n = 1000)

In [34]:
head(dem)

<0 x 0 matrix>

In [35]:
## get followers
gop <- get_followers("FoxNews", n = 1000)

In [36]:
head(gop)

<0 x 0 matrix>

In [37]:
## get followers
mod <- get_followers("cnn", n = 1000)

In [38]:
head(mod)

<0 x 0 matrix>

In [39]:
## get gop users data
gop_data <- lookup_users(gop$user_id)

ERROR: Error in if (x) {: argument is of length zero


In [ ]:
## get dem users data
dem_data <- lookup_users(dem$user_id)

In [ ]:
## get mod users data
mod_data <- lookup_users(mod$user_id)

In [ ]:
## create "partisan" grouping variable
gop_data$partisan <- "gop"
dem_data$partisan <- "dem"
mod_data$partisan <- "mod"

## merge into single data frame
data <- bind_rows(gop_data, dem_data, mod_data)

In [ ]:
## PRINT: group by partisan and summarise friends and followers counts
data %>%
    group_by(partisan) %>%
    summarise(
        friends = mean(friends_count, na.rm = TRUE),
        followers = mean(followers_count, na.rm = TRUE))

In [ ]:
## PLOT: group by partisan and summarise integer vars
data %>%
    mutate(Following = factor(partisan, labels = c("MSNBC", "FoxNews", "CNN"))) %>%
    group_by(Following) %>%
    summarise(
        friends = mean(friends_count, na.rm = TRUE),
        followers = mean(followers_count, na.rm = TRUE)) %>%
    gather(var, count, -Following) %>%
    ggplot(aes(x = Following, y = count, fill = Following)) + 
    geom_col() + 
    facet_wrap(~ var, ncol = 2, scales = "free_y") + 
    theme_bw(base_size = 10) + 
    theme(plot.title = element_text(face = "bold")) + 
    scale_fill_manual(values = c(FoxNews = "red2", MSNBC = "blue", CNN = "purple")) + 
    labs(
        title = "User stats for followers of MSNBC, FoxNews, and CNN",
        subtitle = "Mean number of friends and followers by group (n = 1000)", 
        x = NULL, y = "Count")

In [ ]:
## function to filter and then randomly sample user IDs
rsamp_uq <- function(..., n) {
    
    dots <- list(...)
    
    groups <- names(dots)
    
    user_ids <- lapply(dots, "[[", "user_id")
    
    
    for (i in seq_along(dots)) {
        
        ## filter out users found in other groups and protected users
        dots[[i]] <- filter(
            dots[[i]], 
            !user_id %in% unlist(user_ids[-i]) & !protected
        )
        
        dots[[i]]$group <- groups[i]
    }
    
    sample_users <- function(x, n) x[x$user_id %in% sample(x$user_id, n), ]
        
    out <- lapply(dots, sample_users, n)
        
    bind_rows(out)
}
    
    

## filter and sample 5 from each group
smps <- rsamp_uq(gop = gop_data, dem = dem_data, mod = mod_data, n = 5)

In [ ]:
## get friends for each sampled user
fds <- lapply(smps$user_id, get_friends, verbose = FALSE)

In [ ]:
## add the list of data frames to smps as a list column
smps$fds <- I(fds)

In [ ]:
## create function to identify the top 3 accounts and then lookup their screen names

get_top_names <- function(x, n = 3) {
    
    tbl <- sort(table(unlist(x)), decreasing = TRUE)
    
    if (length(tbl) > n) {
        tbl <- tbl[seq_len(n)]
    }
    
    sns <- lookup_users(names(tbl))
    
    sns$screen_name
}

In [ ]:
## print out top 3 screen names associated with each group
smps %>% 
    group_by(group) %>%
    summarise(
        freq = list(get_top_names(fds))
    )